# How to use the Parent Document Retriever


The ParentDocumentRetriever in LangChain helps balance the need for precise embeddings from smaller document chunks and the context provided by larger documents.

# Why Use Parent Document Retriever?
* The Parent Document Retriever balances two competing goals in document retrieval:

   * Small Chunks for Accurate Embeddings: Smaller chunks yield embeddings that more precisely represent the content of a specific portion of the document.

   * Larger Chunks for Context: Larger chunks preserve the context that might get lost when breaking documents into smaller pieces.

# Key Workflow
Document Loading: Load raw text documents.
Splitting:

Parent Splitting: Split into medium-sized chunks for context.

Child Splitting: Further divide the parent chunks into smaller chunks for embedding storage.

# Storage:
Use a vector store (e.g., Chroma) to store embeddings of smaller chunks.

Use a document store (e.g., InMemoryStore) to maintain parent chunks.

# Retrieval:

Use embeddings to locate relevant small chunks.

Retrieve the corresponding parent documents or larger chunks based on the match.

In [3]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


# Implementation Using Hugging Face Token
Here’s how to use the Parent Document Retriever:

# 1. Setup the Environment
Make sure you have the following dependencies installed:

In [11]:
pip install langchain langchain-chroma langchain-community chromadb


# 2. Load Documents
Load your text documents using the TextLoader:

In [10]:
from langchain_community.document_loaders import TextLoader

# Load text documents
loaders = [
    TextLoader("paul_graham_essay.txt"),
    TextLoader("state_of_the_union.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())


# 3. Create Text Splitters
Define two splitters:

Parent Splitter: For larger chunks.
Child Splitter: For smaller, more granular chunks.

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define the splitters
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)


# 4. Initialize the Vector Store and Document Store
Use the Hugging Face token for the embeddings and set up the storage layers.

In [14]:

from langchain.storage import InMemoryStore
from langchain_chroma import Chroma

# Initialize Hugging Face embeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Use Hugging Face embeddings with your token
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Hugging Face model
    cache_folder="./models",  # Optional: Local cache for models
)

# Setup vector and document stores
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)
store = InMemoryStore()


<ipython-input-14-3ce1a9a3ea96>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/setti

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 5. Initialize the Retriever
Combine all components in the ParentDocumentRetriever:

In [16]:
from langchain.retrievers import ParentDocumentRetriever

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Add documents
retriever.add_documents(docs)


# 6. Test Retrieval
Perform retrieval tasks with your query:

In [17]:
# Search small chunks in the vector store
sub_docs = vectorstore.similarity_search("justice breyer")
print(sub_docs[0].page_content)

# Retrieve larger parent documents
retrieved_docs = retriever.invoke("justice breyer")
print(retrieved_docs[0].page_content)


One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President 

# Key Features to Test
Small Chunk Retrieval: Confirm embeddings yield smaller text segments for precision.

Parent Retrieval: Ensure the retriever returns meaningful, larger contextual chunks.

#Benefits of Using Hugging Face Token
By specifying a Hugging Face model and token, you can:

Avoid dependency on commercial API keys like OpenAI.

Leverage pre-trained transformer-based embeddings.

Maintain full control of local or cloud-based document processing.